In [1]:
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('google_stock_price_test.csv')

In [3]:
df.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2017,778.81,789.63,775.80,786.14,"1,657,300"
1,1/4/2017,788.36,791.34,783.16,786.90,"1,073,000"
2,1/5/2017,786.08,794.48,785.02,794.02,"1,335,200"
3,1/6/2017,795.26,807.90,792.20,806.15,"1,640,200"
4,1/9/2017,806.40,809.97,802.83,806.65,"1,272,400"


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    20 non-null     object 
 1   Open    20 non-null     float64
 2   High    20 non-null     float64
 3   Low     20 non-null     float64
 4   Close   20 non-null     float64
 5   Volume  20 non-null     object 
dtypes: float64(4), object(2)
memory usage: 1.1+ KB


In [5]:
df = df.dropna()  # or df.fillna(df.mean(), inplace=True)

In [6]:
df['Date'] = pd.to_datetime(df['Date'])

In [9]:
df['Volume'] = pd.to_numeric(df['Volume'].replace('[,:\s]', '', regex=True))

In [11]:
df.dtypes

Date      datetime64[ns]
Open             float64
High             float64
Low              float64
Close            float64
Volume             int64
dtype: object

In [16]:
numeric_features = ['Open', 'High', 'Low', 'Close', 'Volume']
scaler = MinMaxScaler()

scaled_data = scaler.fit_transform(df[numeric_features])

# Convert scaled data back into a DataFrame
df_scaled = pd.DataFrame(scaled_data, columns=numeric_features)

# Check the resulting DataFrame
print(df_scaled.head())

       Open      High       Low     Close    Volume
0  0.000000  0.000000  0.000000  0.000000  0.317106
1  0.161864  0.032683  0.143722  0.015344  0.066042
2  0.123220  0.092699  0.180043  0.159095  0.178705
3  0.278814  0.349197  0.320250  0.403998  0.309758
4  0.467627  0.388761  0.527827  0.414092  0.151721


In [17]:
df_scaled = pd.DataFrame(scaled_data, columns=numeric_features)

In [18]:
df_final = pd.concat([df_scaled, df['Close'].reset_index(drop=True)], axis=1)
print(df_final.head())

       Open      High       Low     Close    Volume     Close
0  0.000000  0.000000  0.000000  0.000000  0.317106  0.000000
1  0.161864  0.032683  0.143722  0.015344  0.066042  0.015344
2  0.123220  0.092699  0.180043  0.159095  0.178705  0.159095
3  0.278814  0.349197  0.320250  0.403998  0.309758  0.403998
4  0.467627  0.388761  0.527827  0.414092  0.151721  0.414092


Build a model to predict stock market performance using LSTM
Task
Model Building
Import the necessary Libraries to implement LSTM. Keras deep learning library is a popular one for LSTM. It has extensive functionality and is easy to use.
Run the LSTM algorithm over your train data. Once the model is built, use the test dataset to check the accuracy of the model.
Plot the predicted vs actual data for the predicted duration - The last month (Jan 2017) based on the outputs

In [21]:
import numpy as np
import matplotlib.pyplot as plt 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Sample DataFrame creation - Replace with your actual DataFrame loading
data = {
    'Open': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
    'High': [1.1, 2.1, 3.1, 4.1, 5.1, 6.1, 7.1, 8.1, 9.1, 10.1, 11.1, 12.1, 13.1, 14.1, 15.1, 16.1, 17.1, 18.1, 19.1, 20.1],
    'Low': [0.9, 1.9, 2.9, 3.9, 4.9, 5.9, 6.9, 7.9, 8.9, 9.9, 10.9, 11.9, 12.9, 13.9, 14.9, 15.9, 16.9, 17.9, 18.9, 19.9],
    'Close': [1.05, 2.05, 3.05, 4.05, 5.05, 6.05, 7.05, 8.05, 9.05, 10.05, 11.05, 12.05, 13.05, 14.05, 15.05, 16.05, 17.05, 18.05, 19.05, 20.05],
    'Volume': ['1,000', '2,000', '3,000', '4,000', '5,000', '6,000', '7,000', '8,000', '9,000', '10,000', 
               '11,000', '12,000', '13,000', '14,000', '15,000', '16,000', '17,000', '18,000', '19,000', '20,000']
}
df_final = pd.DataFrame(data)

# Clean the Volume column by removing commas and converting to float
df_final['Volume'] = df_final['Volume'].str.replace(',', '').astype(float)

# Initialize the scaler and apply it to the entire dataset
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df_final[['Open', 'High', 'Low', 'Close', 'Volume']])  # Scale relevant columns

# Check scaled data shape
print("Shape of scaled_data:", scaled_data.shape)

# Split the data into training and test sets
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# Check the sizes of training and testing data
print("Shape of train_data:", train_data.shape)
print("Shape of test_data:", test_data.shape)

# Create sequences with a reduced sequence length
def create_sequences(data, sequence_length=5):  # Change from 60 to 5
    X, y = [], []
    for i in range(sequence_length, len(data)):
        X.append(data[i-sequence_length:i])  # Collect the past 'sequence_length' days
        y.append(data[i, 3])  # Target variable, assuming 'Close' is at index 3
    return np.array(X), np.array(y)

# Generate sequences for training and testing
X_train, y_train = create_sequences(train_data, sequence_length=sequence_length)
X_test, y_test = create_sequences(test_data, sequence_length=sequence_length)

# Check the shapes of the sequences
print("Shape of X_train:", X_train.shape)  # Expecting (samples, timesteps, features)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

# Check for empty sequences
if X_train.size == 0 or y_train.size == 0:
    print("No training sequences were created. Check the input data and sequence length.")

# Ensure that X_train has the correct dimensions for LSTM
if X_train.size > 0:
    print("Number of features in X_train:", X_train.shape[2])

# If necessary, reshape X_train and X_test to ensure they are 3D
if X_train.shape[0] > 0 and X_train.shape[1] > 0:
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
if X_test.shape[0] > 0 and X_test.shape[1] > 0:
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])

# Build LSTM Model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, batch_size=1, epochs=5)  # Adjust epochs and batch size as needed


Shape of scaled_data: (20, 5)
Shape of train_data: (16, 5)
Shape of test_data: (4, 5)
Shape of X_train: (0,)
Shape of y_train: (0,)
Shape of X_test: (0,)
Shape of y_test: (0,)
No training sequences were created. Check the input data and sequence length.


IndexError: tuple index out of range

In [41]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# First, let's verify the file exists and check its size
print("File exists:", os.path.exists('Google_Stock_Price_Train.csv'))
print("File size:", os.path.getsize('Google_Stock_Price_Train.csv'), "bytes")

# Set a higher recursion limit
sys.setrecursionlimit(10000)

# Read the CSV file with minimal processing
df = pd.read_csv('Google_Stock_Price_Train.csv', encoding='utf-8')

# Basic data info
print("\
Dataset Shape:", df.shape)
print("\
Columns:", df.columns.tolist())
print("\
First 5 rows:")
print(df.head())

File exists: True
File size: 63488 bytes
Dataset Shape: (1258, 6)
Columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
First 5 rows:
       Date    Open    High     Low   Close      Volume
0  1/3/2012  325.25  332.83  324.97  663.59   7,380,500
1  1/4/2012  331.27  333.87  329.08  666.45   5,749,400
2  1/5/2012  329.83  330.75  326.89  657.21   6,590,300
3  1/6/2012  328.34  328.77  323.68  648.24   5,405,900
4  1/9/2012  322.04  322.29  309.46  620.76  11,688,800
